In [4]:
##############################################################
##                   SPASSO                  vs 0.0.0beta  ###
##############################################################
##  This  file  is  free software ;  it  is distributed  in  #
##  the hope that  it  will  be  useful,  but  without  any  #
##  warranty.  You  can  redistribute  it and/or modify  it  #
##  under  the  terms  of  the  GNU  General Public License  #
##  as  published  by  the   Free  Software  Foundation  at  #
##       http://www.gnu.org/copyleft/gpl.html                #
##############################################################
#                  MAIN_OCEANCOLOR.py
#
# Python script for convertion of oceancolor CHL data
# Input: CMEMS *_CHL*.nc files
#
# Output: .mat file containing Longitude (-180 to 180) for mediterranean products, Latitude and SST[degC] 
# Output: .mat file containing Longitude (0 to 360) for global products, Latitude and SST[degC] 
##############################################################
# Credits: F.d'Ovidio
#          F.Nencioli
#          L.Rousselet
#          A.Doglioli
#          Alice Della Penna
#          Stephanie Barrillon
#          Anne Petrenko
#          Anais Ricout
###############################################################
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os 
import sys 
import glob
from pathlib import Path
from netCDF4 import Dataset
import cmocean as cm_oc
import colormaps as cmaps
import csv
import matplotlib.rcsetup as rcsetup
import numpy as np
import scipy.io as sio 
import matplotlib.pyplot as plt
import xarray as xr
import matplotlib as mpl 
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import matplotlib 
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

home = str(Path.home())
os.environ['PROJ_LIB'] = "{ os.environ['CONDA_PREFIX'] }/share/proj"
from mpl_toolkits.basemap import Basemap  #deprecated ?

from matplotlib.patches import Polygon
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cftime import utime
#import cftime
#import nc_time_axis
import time
import datetime 
#############################################
args=sys.argv
### TEST MODE 
#if len(args)==1:
#        dir_wrk='/home/gcambon/MYGITHUB/SPASSO/Cruises/DEMO/Wrk'
#        dir_cruise='/home/gcambon/MYGITHUB/SPASSO/Cruises/DEMO'
#        ### OPERATIONAL MODE
#else:
#        dir_wrk=sys.argv[1]
#        dir_cruise=sys.argv[2]
#        ### END if len(args)<1:

dir_wrk='/net/spica/local/tmp/3/gcambon/SPASSO/SPASSO.beta/Cruises/DEMO/Wrk'
dir_cruise='/home/gcambon/MYGITHUB/SPASSO/Cruises/DEMO'

# to load cruise configuration
exec(open(dir_cruise + "/domain_limits.py").read())

######## CHL FILES #####################

In [5]:
dir_cruise
dir_wrk

'/net/spica/local/tmp/3/gcambon/SPASSO/SPASSO.beta/Cruises/DEMO/Wrk'

In [6]:
### L4 MED 1KM resolution
#filelist=glob.glob(dir_wrk+'/*_d-OC_CNR-L4-CHL-INTERP_MULTI_1KM-MED-NRT-v02.nc')
filelist=glob.glob(dir_wrk+'/*_d-ACRI-L4-CHL-MULTI_4KM-GLO-NRT.nc')
print(filelist)
for nc_name in filelist:
        filemat=nc_name[0:-2]+'mat'
        print(filemat)
        #  Load netcdf data
        ncdata=xr.open_dataset(nc_name,decode_times='False')
        
        #  extract for the domain
        data_domain = ncdata.sel(lon=slice(Lon[0],Lon[1]), lat=slice(Lat[0],Lat[1]))

['/net/spica/local/tmp/3/gcambon/SPASSO/SPASSO.beta/Cruises/DEMO/Wrk/20220122_d-ACRI-L4-CHL-MULTI_4KM-GLO-NRT.nc']
/net/spica/local/tmp/3/gcambon/SPASSO/SPASSO.beta/Cruises/DEMO/Wrk/20220122_d-ACRI-L4-CHL-MULTI_4KM-GLO-NRT.mat


In [7]:
ncdata.time

<xarray.DataArray 'time' (time: 1)>
array([cftime.DatetimeGregorian(2022, 1, 22, 0, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 2022-01-22 00:00:00
Attributes:
    standard_name:  time
    long_name:      reference time
    axis:           T
    valid_min:      0.0
    valid_max:      401767.0

In [8]:
        #  save the data in a mat file
        output = {'lon':data_domain['lon'].values, 
                  'lat':data_domain['lat'].values,
                  'time':data_domain['time'].values,
                  'Chl':np.squeeze(data_domain['CHL'].values)}

In [9]:
sio.matlab.savemat(filemat, output)

TypeError: Could not convert 2022-01-22 00:00:00 (type <class 'cftime._cftime.DatetimeGregorian'>) to array

In [ ]:
sio.matlab.savemat?

In [ ]:
sio.matlab.savemat(filemat, output)

In [ ]:
output?

In [ ]:
        sio.matlab.savemat(filemat, output)

In [ ]:
### L3 MED MULTI 1KM resolution
files=glob.glob(dir_wrk+'/*_d-OC_CNR-L3-CHL-MedOC4AD4_MULTI_1KM-MED-NRT-v02.nc'); 
for nc_name in files:
        filemat=nc_name[0:-2]+'mat'
        
        #  Load netcdf data
        ncdata=xr.open_dataset(nc_name,decode_times='False')
        
        #  extract for the domain
        data_domain = ncdata.sel(lon=slice(Lon[0],Lon[1]), lat=slice(Lat[0],Lat[1]))
        
        #  save the data in a mat file
        output = {'lon':data_domain['lon'].values, 'lat':data_domain['lat'].values, 'time':data_domain['time'].values, 'Chl':data_domain['CHL'].values}
        
        sio.matlab.savemat(filemat, output)

        
### L3 MED OLCI_a 1KM resolution
files=glob.glob(dir_wrk+'/*_d-OC_CNR-L3-CHL-MedOC4Ad4_Oa_1KM-MED-NRT-v02.nc'); 
for nc_name in files:
        filemat=nc_name[0:-2]+'mat'
        
        #  Load netcdf data
        ncdata=xr.open_dataset(nc_name,decode_times='False')
        
        #  extract for the domain
        data_domain = ncdata.sel(lon=slice(Lon[0],Lon[1]), lat=slice(Lat[0],Lat[1]))
        
        #  save the data in a mat file
        output = {'lon':data_domain['lon'].values, 'lat':data_domain['lat'].values, 'time':data_domain['time'].values, 'Chl':data_domain['CHL'].values}
        
        sio.matlab.savemat(filemat, output)
        

### L3 GLOBAL MULTI 4km resolution
files=glob.glob(dir_wrk+'/*_d-ACRI-L3-CHL-MULTI_4KM-GLO-NRT-v02.nc');
for nc_name in files:
        filemat=nc_name[0:-2]+'mat'

        #  Load netcdf data
        ncdata=xr.open_dataset(nc_name,decode_times='False')

        #  extract for the domain
        data_domain = ncdata.sel(lon=slice(Lon[0],Lon[1]), lat=slice(Lat[1],Lat[0]))

        #  save the data in a mat file
        output = {'lon':data_domain['lon'].values, 'lat':np.flipud(data_domain['lat'].values), 'time':data_domain['time'].values, 'Chl':np.fliplr(data_domain['CHL'].values)}

        sio.matlab.savemat(filemat, output)
        
        
### L3 GLOBAL OLCI_a 4km resolution
files=glob.glob(dir_wrk+'/*_d-ACRI-L3-CHL-AV_Oa_4KM-GLO-NRT-v02.nc'); 
for nc_name in files:
        filemat=nc_name[0:-2]+'mat'
        
        #  Load netcdf data
        ncdata=xr.open_dataset(nc_name,decode_times='False')        
        
        #  extract for the domain
        data_domain = ncdata.sel(lon=slice(Lon[0],Lon[1]), lat=slice(Lat[1],Lat[0]))
        
        #  save the data in a mat file
        output = {'lon':data_domain['lon'].values, 'lat':np.flipud(data_domain['lat'].values), 'time':data_domain['time'].values, 'Chl':np.fliplr(data_domain['CHL'].values)}
        
        sio.matlab.savemat(filemat, output)